# iQuHACK 2026 - Clifford+T Circuit Compilation

This notebook optimizes 11 unitaries into Clifford+T circuits, minimizing T-gate count while maximizing fidelity.

**Pipeline**:
1. KAK/Euler decomposition → Clifford+T conversion via manual identities
2. Reed-Muller synthesis (rmsynth) for T-count optimization
3. Brute force search fallback for low-fidelity circuits

In [ ]:
# Import Required Libraries
import numpy as np
from scipy.linalg import polar
import os
import sys
import importlib

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, process_fidelity

# Import the optimizer module and reload to pick up any changes
import optimize_unitaries
importlib.reload(optimize_unitaries)

# Import the optimizer functions (includes brute force analyzer functions)
from optimize_unitaries import (
    load_unitary, decompose_and_optimize, apply_manual_identities, optimize_circuit,
    optimize_diagonal_chunks_with_rmsynth,
    # Brute force analyzer functions
    analyze_unitary, brute_force_search, quick_search, compare_circuits, full_analysis,
    # Main helper function (same as defined locally but imported to ensure consistency)
    optimize_and_display
)

# For nice matrix display
np.set_printoptions(precision=4, suppress=True, linewidth=120)

## Helper Function

The `optimize_and_display` function is imported from `optimize_unitary.py` to ensure the notebook and Python script produce identical outputs.

It:
1. Loads the unitary from `.npy` file
2. Runs the standard optimization pipeline
3. Tries brute force search if fidelity < 0.95 (2-qubit only)
4. Displays T-count, CX count, distance, fidelity, and circuit

**Note**: To run the same optimization from the command line, use:
```bash
python optimize_unitary.py --batch  # Process all 11 unitaries
python optimize_unitary.py unitary1.npy output.qasm --effort 3  # Single file
```

---
## Unitary 1

In [23]:
# Unitary 1
qc1, U1 = optimize_and_display("Unitary 1", "unitary1.npy", verbose=False)

Unitary 1

Method: Standard optimizer
TOTAL T-GATES: 0
CX Count: 1
Total Gates: 3
Gate counts: {'sdg': 1, 'cx': 1, 's': 1}
Operator Norm Distance: 0.000000e+00
Process Fidelity: 1.000000

Circuit:
     ┌─────┐┌───┐┌───┐
q_0: ┤ Sdg ├┤ X ├┤ S ├
     └─────┘└─┬─┘└───┘
q_1: ─────────■───────
                      



---
## Unitary 2

In [6]:
# Unitary 2
qc2, U2 = optimize_and_display("Unitary 2", "unitary2.npy", verbose=False)

Unitary 2

Method: Standard optimizer
TOTAL T-GATES: 0
CX Count: 2
Total Gates: 2
Gate counts: {'cx': 2}
Operator Norm Distance: 2.239290e-01
Process Fidelity: 0.975085

Circuit:
     ┌───┐┌───┐
q_0: ┤ X ├┤ X ├
     └─┬─┘└─┬─┘
q_1: ──■────■──
               



---
## Unitary 3

In [7]:
# Unitary 3
qc3, U3 = optimize_and_display("Unitary 3", "unitary3.npy", verbose=False)

Unitary 3

Method: Standard optimizer
TOTAL T-GATES: 1
CX Count: 2
Total Gates: 3
Gate counts: {'cx': 2, 'tdg': 1}
Operator Norm Distance: 5.609251e-02
Process Fidelity: 0.996856

Circuit:
global phase: π/8
                      
q_0: ──■───────────■──
     ┌─┴─┐┌─────┐┌─┴─┐
q_1: ┤ X ├┤ Tdg ├┤ X ├
     └───┘└─────┘└───┘



---
## Unitary 4

In [8]:
# Unitary 4
qc4, U4 = optimize_and_display("Unitary 4", "unitary4.npy", verbose=False)

Unitary 4

  [Trying brute force search - fidelity 0.8092 < 0.95]

BRUTE FORCE SEARCH (T≤3, CX≤2, timeout=60s)

Searching with 1 CX...
  1280 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.1647
  T=0, CX=1, fid=0.1808
  T=2, CX=1, fid=0.1838
  T=0, CX=1, fid=0.2029
  T=0, CX=1, fid=0.2500

Searching with 2 CX...
  41984 valid gate combos (of 46656 total)
  T=0, CX=2, fid=0.3616
  T=1, CX=2, fid=0.4046
  T=1, CX=2, fid=0.6173
  T=2, CX=2, fid=0.6907
  T=0, CX=2, fid=0.7232
  T=1, CX=2, fid=0.8092

Done: 170496 circuits in 5.6s (30317/sec)

Best: T=1, fid=0.809193
     ┌───┐                 ┌───┐
q_0: ┤ H ├──■───────────■──┤ H ├
     ├───┤┌─┴─┐┌─────┐┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ H ├
     └───┘└───┘└─────┘└───┘└───┘

Method: Standard optimizer
TOTAL T-GATES: 1
CX Count: 2
Total Gates: 7
Gate counts: {'h': 4, 'cx': 2, 'tdg': 1}
Operator Norm Distance: 4.995530e-01
Process Fidelity: 0.809193

Circuit:
global phase: π/8
     ┌───┐                 ┌───┐
q_0: ┤ H ├──■─────────

---
## Unitary 5

In [9]:
# Unitary 5
qc5, U5 = optimize_and_display("Unitary 5", "unitary5.npy", verbose=False)

Unitary 5

Method: Standard optimizer
TOTAL T-GATES: 0
CX Count: 3
Total Gates: 7
Gate counts: {'h': 4, 'cx': 3}
Operator Norm Distance: 4.454693e-16
Process Fidelity: 1.000000

Circuit:
global phase: π/4
          ┌───┐     ┌───┐     
q_0: ──■──┤ H ├──■──┤ H ├──■──
     ┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐
q_1: ┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├
     └───┘└───┘└───┘└───┘└───┘



---
## Unitary 6

In [10]:
# Unitary 6
qc6, U6 = optimize_and_display("Unitary 6", "unitary6.npy", verbose=False)

Unitary 6

  [Trying brute force search - fidelity 0.8229 < 0.95]

BRUTE FORCE SEARCH (T≤4, CX≤2, timeout=60s)

Searching with 1 CX...
  1296 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.1649
  T=1, CX=1, fid=0.1695
  T=0, CX=1, fid=0.1708
  T=2, CX=1, fid=0.1914
  T=1, CX=1, fid=0.1986
  T=3, CX=1, fid=0.1991
  T=2, CX=1, fid=0.2017
  T=2, CX=1, fid=0.2032
  T=0, CX=1, fid=0.3452
  T=1, CX=1, fid=0.3769

Searching with 2 CX...
  45824 valid gate combos (of 46656 total)
  T=2, CX=2, fid=0.3965
  T=1, CX=2, fid=0.5398
  T=2, CX=2, fid=0.5734
  T=0, CX=2, fid=0.6324
  T=1, CX=2, fid=0.6717
  T=1, CX=2, fid=0.7525
  T=2, CX=2, fid=0.8229

Done: 185888 circuits in 6.1s (30589/sec)

Best: T=2, fid=0.822930
      ┌───┐      ┌─────┐     ┌─────┐
q_0: ─┤ S ├───■──┤ Sdg ├──■──┤ Tdg ├
     ┌┴───┴┐┌─┴─┐└─────┘┌─┴─┐└┬───┬┘
q_1: ┤ Sdg ├┤ X ├───────┤ X ├─┤ T ├─
     └─────┘└───┘       └───┘ └───┘ 
  [Brute force found better: F=0.8229]

Method: Brute force search
TOTAL T-GATES: 2
CX Count: 2


---
## Unitary 7

In [11]:
# Unitary 7
qc7, U7 = optimize_and_display("Unitary 7", "unitary7.npy", verbose=False)

Unitary 7

  [Trying brute force search - fidelity 0.6950 < 0.95]

BRUTE FORCE SEARCH (T≤5, CX≤2, timeout=60s)

Searching with 1 CX...
  1296 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.0659
  T=0, CX=1, fid=0.0931
  T=1, CX=1, fid=0.1219
  T=0, CX=1, fid=0.1532
  T=1, CX=1, fid=0.2308
  T=0, CX=1, fid=0.2502
  T=2, CX=1, fid=0.2566
  T=1, CX=1, fid=0.2710
  T=0, CX=1, fid=0.2802
  T=0, CX=1, fid=0.3719
  T=1, CX=1, fid=0.3761
  T=1, CX=1, fid=0.4333

Searching with 2 CX...
  46592 valid gate combos (of 46656 total)
  T=0, CX=2, fid=0.4344
  T=1, CX=2, fid=0.4380
  T=3, CX=2, fid=0.4559
  T=3, CX=2, fid=0.4625
  T=2, CX=2, fid=0.4886
  T=1, CX=2, fid=0.4972
  T=1, CX=2, fid=0.5416
  T=2, CX=2, fid=0.6101
  T=2, CX=2, fid=0.6116
  T=3, CX=2, fid=0.6950

Done: 188960 circuits in 6.2s (30454/sec)

Best: T=3, fid=0.694964
      ┌───┐      ┌─────┐     ┌─────┐
q_0: ─┤ S ├───■──┤ Sdg ├──■──┤ Tdg ├
     ┌┴───┴┐┌─┴─┐├─────┤┌─┴─┐└┬───┬┘
q_1: ┤ Sdg ├┤ X ├┤ Tdg ├┤ X ├─┤ T ├─
     └─────┘└

---
## Unitary 8

In [12]:
# Unitary 8
qc8, U8 = optimize_and_display("Unitary 8", "unitary8.npy", verbose=False)

Unitary 8

Method: Standard optimizer
TOTAL T-GATES: 3
CX Count: 5
Total Gates: 10
Gate counts: {'cx': 5, 'h': 2, 't': 2, 'tdg': 1}
Operator Norm Distance: 4.998217e-16
Process Fidelity: 1.000000

Circuit:
global phase: 0
                           ┌───┐┌───┐     ┌───┐     
q_0: ───────■───────────■──┤ T ├┤ H ├──■──┤ X ├──■──
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐├───┤└───┘┌─┴─┐└─┬─┘┌─┴─┐
q_1: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─────┤ X ├──■──┤ X ├
     └───┘└───┘└─────┘└───┘└───┘     └───┘     └───┘



---
## Unitary 9

In [13]:
# Unitary 9
qc9, U9 = optimize_and_display("Unitary 9", "unitary9.npy", verbose=False)

Unitary 9

  [Trying brute force search - fidelity 0.6250 < 0.95]

BRUTE FORCE SEARCH (T≤2, CX≤2, timeout=60s)

Searching with 1 CX...
  1152 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.1562
  T=1, CX=1, fid=0.1821
  T=1, CX=1, fid=0.2667
  T=2, CX=1, fid=0.3109
  T=0, CX=1, fid=0.3125
  T=1, CX=1, fid=0.3643

Searching with 2 CX...
  31744 valid gate combos (of 46656 total)
  T=0, CX=2, fid=0.6250
  T=1, CX=2, fid=0.7286

Done: 129280 circuits in 4.4s (29649/sec)

Best: T=1, fid=0.728553
     ┌───┐     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├──■──┤ S ├
     ├───┤┌─┴─┐├───┤┌─┴─┐├───┤
q_1: ┤ H ├┤ X ├┤ H ├┤ X ├┤ T ├
     └───┘└───┘└───┘└───┘└───┘
  [Brute force found better: F=0.7286]

Method: Brute force search
TOTAL T-GATES: 1
CX Count: 2
Total Gates: 8
Gate counts: {'h': 4, 'cx': 2, 's': 1, 't': 1}
Operator Norm Distance: 1.832237e+00
Process Fidelity: 0.156250

Circuit:
global phase: 0.32175
     ┌───┐     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ H ├──■──┤ S ├
     ├───┤┌─┴─┐├───┤┌─┴─┐├

---
## Unitary 10

In [14]:
# Unitary 10
qc10, U10 = optimize_and_display("Unitary 10", "unitary10.npy", verbose=False)

Unitary 10

  [Trying brute force search - fidelity 0.5661 < 0.95]

BRUTE FORCE SEARCH (T≤2, CX≤2, timeout=60s)

Searching with 1 CX...
  1152 valid gate combos (of 1296 total)
  T=0, CX=1, fid=0.0454
  T=0, CX=1, fid=0.0724
  T=0, CX=1, fid=0.1928
  T=1, CX=1, fid=0.2105
  T=1, CX=1, fid=0.2438
  T=0, CX=1, fid=0.2624
  T=1, CX=1, fid=0.2860
  T=1, CX=1, fid=0.2920
  T=0, CX=1, fid=0.3214

Searching with 2 CX...
  31744 valid gate combos (of 46656 total)
  T=1, CX=2, fid=0.3703
  T=0, CX=2, fid=0.4014
  T=1, CX=2, fid=0.4217
  T=1, CX=2, fid=0.4448
  T=0, CX=2, fid=0.4762
  T=1, CX=2, fid=0.4831

Done: 129280 circuits in 4.3s (30065/sec)

Best: T=1, fid=0.483103
     ┌─────┐┌───┐┌───┐┌───┐┌───┐
q_0: ┤ Tdg ├┤ X ├┤ H ├┤ X ├┤ S ├
     └┬───┬┘└─┬─┘└───┘└─┬─┘├───┤
q_1: ─┤ H ├───■─────────■──┤ H ├
      └───┘                └───┘

Method: Standard optimizer
TOTAL T-GATES: 0
CX Count: 0
Total Gates: 4
Gate counts: {'h': 2, 'z': 2}
Operator Norm Distance: 1.089767e+00
Process Fidelity: 0.5660

---
## Unitary 11

In [15]:
# Unitary 11
qc11, U11 = optimize_and_display("Unitary 11", "unitary11.npy", verbose=False)

Unitary 11

Method: Standard optimizer
TOTAL T-GATES: 4
CX Count: 10
Total Gates: 14
Gate counts: {'cx': 10, 'tdg': 4}
Operator Norm Distance: 3.550831e-16
Process Fidelity: 1.000000

Circuit:
global phase: 2π
                                                                                   
q_0: ──■────────────────■─────────■────────────────────────────────────────■───────
       │                │         │                                        │       
q_1: ──┼────■───────────┼────■────┼────■───────────────────────■───────────┼───────
     ┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐  │    │                       │           │       
q_2: ┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ X ├──┼────┼───────────■───────────┼───────────┼────■──
     └───┘└───┘└─────┘└───┘└───┘┌─┴─┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─────┐┌─┴─┐┌─┴─┐
q_3: ───────────────────────────┤ X ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ Tdg ├┤ X ├┤ X ├
                                └───┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└───┘



---
## Complete Verification Summary

All 11 unitaries processed with the following metrics:
- **T-Count**: Number of T and Tdg gates (primary optimization target)
- **CX Count**: Number of CNOT gates
- **Total Gates**: Sum of all gates in the circuit  
- **Distance**: Operator norm distance ‖U_target - U_circuit‖
- **Fidelity**: Process fidelity between circuit and target
- **Score**: Competition metric = T-count + 200 × (1 - Fidelity)²

In [16]:
# Complete Verification Summary Table
import pandas as pd

# Collect all results
results = []
unitaries = [
    ("Unitary 1", qc1, U1),
    ("Unitary 2", qc2, U2),
    ("Unitary 3", qc3, U3),
    ("Unitary 4", qc4, U4),
    ("Unitary 5", qc5, U5),
    ("Unitary 6", qc6, U6),
    ("Unitary 7", qc7, U7),
    ("Unitary 8", qc8, U8),
    ("Unitary 9", qc9, U9),
    ("Unitary 10", qc10, U10),
    ("Unitary 11", qc11, U11),
]

for name, qc, U in unitaries:
    ops = qc.count_ops()
    t_count = ops.get('t', 0) + ops.get('tdg', 0)
    cx_count = ops.get('cx', 0)
    total_gates = sum(ops.values())
    U_final = Operator(qc).data
    dist = np.linalg.norm(U - U_final, ord=2)
    fid = process_fidelity(Operator(U), Operator(qc))
    n_qubits = int(np.log2(U.shape[0]))
    
    # Calculate score: T-count + 200 * (1 - Fidelity)^2
    score = t_count + 200 * (1 - fid) ** 2
    
    results.append({
        "Unitary": name,
        "Qubits": n_qubits,
        "T-Count": t_count,
        "CX": cx_count,
        "Total": total_gates,
        "Distance": f"{dist:.2e}",
        "Fidelity": f"{fid:.6f}",
        "Score": f"{score:.2f}",
    })

df = pd.DataFrame(results)
print("=" * 100)
print("COMPLETE VERIFICATION SUMMARY - All 11 Unitaries")
print("Score = T-count + 200 × (1 - Fidelity)²")
print("=" * 100)
print(df.to_string(index=False))
print("=" * 100)

# Summary statistics
total_t = sum(r['T-Count'] for r in results)
total_cx = sum(r['CX'] for r in results)
avg_fid = sum(float(r['Fidelity']) for r in results) / len(results)
total_score = sum(float(r['Score']) for r in results)

print(f"\n📊 TOTALS:")
print(f"   Total T-Gates: {total_t}")
print(f"   Total CX Gates: {total_cx}")
print(f"   Average Fidelity: {avg_fid:.6f}")
print(f"   Total Score: {total_score:.2f}")
print()

# Flag circuits that might need attention
low_fid = [(r['Unitary'], float(r['Fidelity'])) for r in results if float(r['Fidelity']) < 0.9]
if low_fid:
    print("⚠️  Circuits with fidelity < 0.9:")
    for name, fid in low_fid:
        print(f"   - {name}: F={fid:.4f}")
else:
    print("✅ All circuits have fidelity ≥ 0.9")

COMPLETE VERIFICATION SUMMARY - All 11 Unitaries
Score = T-count + 200 × (1 - Fidelity)²
   Unitary  Qubits  T-Count  CX  Total Distance Fidelity  Score
 Unitary 1       2        0   1      3 0.00e+00 1.000000   0.00
 Unitary 2       2        0   2      2 2.24e-01 0.975085   0.12
 Unitary 3       2        1   2      3 5.61e-02 0.996856   1.00
 Unitary 4       2        1   2      7 5.00e-01 0.809193   8.28
 Unitary 5       2        0   3      7 4.45e-16 1.000000   0.00
 Unitary 6       2        2   2      7 4.45e-01 0.822930   8.27
 Unitary 7       2        3   2      8 8.51e-01 0.694964  21.61
 Unitary 8       2        3   5     10 5.00e-16 1.000000   3.00
 Unitary 9       2        1   2      8 1.83e+00 0.156250 143.38
Unitary 10       2        0   0      4 1.09e+00 0.566066  37.66
Unitary 11       4        4  10     14 3.55e-16 1.000000   4.00

📊 TOTALS:
   Total T-Gates: 15
   Total CX Gates: 31
   Average Fidelity: 0.820122
   Total Score: 227.32

⚠️  Circuits with fidelity < 0.9:
 